# VGG-16 (Transfer Learning)

In [1]:
import torch
import torchvision.models as models
from torch import nn, optim
from torchsummary import summary
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from ptflops import get_model_complexity_info
import os

# Import custom utility module
from utils import get_transform, prepare_train_test_data

In [2]:
# Device configuration (Select CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n Device : {device}")


 Device : cuda


### Load and prepare the data

In [3]:
# Transform data
image_input_size = (3, 224, 224) # Image input size for the model
transform = get_transform(resize_image_size = (224, 224))

# Prepare Train/Val/Test Data
train_loader, val_loader, test_loader = prepare_train_test_data(transform)

### Load pre-trained VGG-16 model and modify it

In [4]:
# Load pre-trained VGG-16
model = models.vgg16(pretrained=True)

# Modify the final fully connected layer for binary classification
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 2)  # Binary classification: Internal waves (1) or No waves (0)
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
# optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.1) # Implement a learning rate scheduler to adjust the learning rate during training

# Print summary
summary(model, input_size=image_input_size)

C:\Users\ishar\.virtualenvs\ML_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ishar\.virtualenvs\ML_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

### Train the model

In [7]:
# Initialize a DataFrame to log training/validation metrics
log_df = pd.DataFrame(columns=["Epoch", "Train Loss", "Validation Loss", "Validation Accuracy", "FLOPs"])

# Check if a checkpoint exists to resume training
checkpoint_path = "model_outputs_data/vgg16_checkpoint.pth"
start_epoch = 0
best_val_loss = float('inf')

if os.path.exists(checkpoint_path):
    print("Loading checkpoint...")
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    best_val_loss = checkpoint['best_val_loss']
    # Load training logs
    log_df = pd.read_csv("model_outputs_data/training_vgg16_logs.csv")
    print(f"Resuming training from epoch {start_epoch}")
else:
    print("Starting training from scratch.")

# Training loop
num_epochs = 50  # Total number of epochs

for epoch in range(start_epoch, num_epochs):
    # Training phase
    model.train()
    train_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = val_correct / val_total * 100

    # Adjust learning rate
    scheduler.step(val_loss)

    # Calculate FLOPs after each epoch
    with torch.no_grad():
        flops, _ = get_model_complexity_info(model, image_input_size, as_strings=False, print_per_layer_stat=False)

    # Log metrics into the DataFrame
    new_row = {
        "Epoch": epoch + 1,
        "Train Loss": train_loss / len(train_loader),
        "Validation Loss": val_loss / len(val_loader),
        "Validation Accuracy": val_accuracy,
        "FLOPs": flops
    }
    log_df = pd.concat([log_df, pd.DataFrame([new_row])], ignore_index=True)

    # Save training logs after each epoch
    log_df.to_csv("model_outputs_data/training_vgg16_logs.csv", index=False)

    # Print metrics for this epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss / len(train_loader):.4f}, "
          f"Val Loss: {val_loss / len(val_loader):.4f}, Val Accuracy: {val_accuracy:.2f}%, "
          f"GFLOPs: {(new_row['FLOPs'] / 1e9):.2f}")

    # Save the best model based on validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "model_outputs_data/best_vgg16_model.pth")

    # Save checkpoint after each epoch
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'best_val_loss': best_val_loss,
    }, checkpoint_path)

# Remove the checkpoint file after training is complete
if os.path.exists(checkpoint_path):
    os.remove(checkpoint_path)


Starting training from scratch.


KeyError: 'momentum'

### Prediction and Model evaluation

In [ ]:
# Test the model
# model.load_state_dict(torch.load("model_outputs_data/best_vgg16_model.pth"))  # Load the best model
model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Compute Evaluation Metrics
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='binary')
recall = recall_score(all_labels, all_preds, average='binary')
f1 = f1_score(all_labels, all_preds, average='binary')

print(f"\nTest Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=["No Internal Waves", "Internal Waves"]))

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(all_labels, all_preds))

In [ ]:
# Save prediction data to a CSV file (As a backup)
results_df = pd.DataFrame({
    "true_labels": all_labels,
    "predicted_labels": all_preds
})
results_df.to_csv("model_outputs_data/model_prediction_logs/vgg16_labels_predictions.csv", index=False)